![MOSEK ApS](https://mosek.com/files/mosek_logo_color.png )


`Fusion`: Object-Oriented API for Conic Optimization
==============================

`Fusion` is an Object-Oriented API specifically designed for for Conic Optimization. Its main features are:
  
* (almost) Seamlessly multi-language (`C#, Java, Python, Matlab`)

* Minimalistic and Safe

* Extremely Disciplined Conic Optimization

* Limited overhead

* **MOSEK** gets what you write!



Basic Idea
-----------------

We want to introduce an *object-oriented* interface for optimization problem of the form:

\begin{equation}
  \begin{array}{lll}
    \min & \sum_j \langle C_j , X_j \rangle&\\
    s.t.&&\\
    & \sum_j \langle A_{ij},  X_j \rangle+ B_i \in \mathcal{K}_i^{n_i} & i=1,\ldots,k\\
  \end{array}
\end{equation}


Where $A_{ij},B_i$ and $C_j$ are matrices of suitable dimensions, $X_j$ are bidimensional variables. Cones $\mathcal{K}_i^{n_i}$ are  

* $\mathbb{R}_+$ - positive orthant ,

* $\mathcal{Q}$ - Lorentz cone,

* $\mathcal{Q_r}$ - Rotated quadratic cone.

Dimensions must match accordingly.

To express this kind of problem we need only these *basic constructs*:

    obj. fun. := ( linear expression , sense)
    variable  := ( dimension , domain)
    constraint:= ( linear expression , domain)


Linear expressions need:

* linear operators
* matrices
* variables


A single class to rule them all: ``Model``
-------------------------------------------

It is the object the user interfaces with:

* Model building:

        var  = M.variable( [name], dimension, domain)
        cons = M.constraint( [name], linear expression , domain)
        M.objective( [name], sense, linear expression)
  
  
  
* Solver access:
  
        M.solve()
  
        print M.getPrimalSolutionStatus()
  
 

**Note: each model owns the objects it builds!**



Example: a simple conic quadratic problem
-----------------------------------------


Assume we want to find the solution of system of linear equality and inequalities that is as close as possible to a given vector $y$. The formal definition is:

\begin{equation}
    \begin{aligned}
    & \min \|x - y\|_2 \\
    & s.t.\\
    & Ax = b \\
    & l_x\leq x \leq u_x
    \end{aligned}
\end{equation}

In conic form it reads

\begin{equation}
    \begin{aligned}
    & \min t \\
    & s.t.\\
    & (t, x - y) \in \mathcal{Q}^{n+1}\\
    & Ax = b \\
    & l_x\leq x \leq u_x
    \end{aligned}
\end{equation}


First, let us declare some input data:

In [32]:
n = 5

y = [0. for i in range(n)]

A = [ [12., 20.,  0., 25., 15.],\
      [ 2.,  8., 16.,  0.,  0.],\
      [20., 20., 20., 20., 20.]]
     
b = [288.,192.,384.]

lx= [0. for i in range(n)]
ux= [1000. for i in range(n)]


A `Fusion` code starts with the model declaration:

In [33]:
from mosek.fusion import *

M = Model('My test')

then we input the $x$ variables

In [34]:
x = M.variable('x', n, Domain.inRange(lx,ux))

and $t$

In [35]:
t = M.variable('t', 1, Domain.unbounded())

The objective function follows

In [36]:
M.objective(ObjectiveSense.Minimize, t)

We can declare

* the conic quadratic constraint stacking $t$ and $x$ together:
*  Note how we can store a reference to the constraints.


In [37]:
M.constraint(Expr.vstack(t, Expr.sub(x,y)), Domain.inQCone())
le = M.constraint( Expr.mul(A,x), Domain.equalsTo(b))


Note how

* the cone dimensions are deduced automatically from the input.
* we can store a reference to the constraints.

Now we only need to run the solver and print the solution:

In [38]:
M.solve()

print 'primal solution status = ',M.getPrimalSolutionStatus()
print 'primal solution        = \n', x.level()

primal solution status =  SolutionStatus.Optimal
primal solution        = 
[-1.2462112350726904e-08, 14.399999925447373, 4.800000005447712, 4.1706478408768495e-08, -1.3557713255520233e-08]


Information about the dual can be obtained directly:

In [39]:
print 'dual solution status = ', M.getDualSolutionStatus()
print 'Ax-b dual mult.      = ', le.dual()

dual solution status =  SolutionStatus.Optimal
Ax-b dual mult.      =  [0.14471806520291086, 0.2827359317801117, -0.2103780441985366]


Notes
----------

1. The order in which the model is declared is not important, i.e. constraints, variables and the objective functions can be declared in any order, as long as it makes sense.

2. `Fusion` provides only a (large) subset of functionalities of the low-level Optimizer API.


Conclusion
--------------------


---------------------------------------------------------------------------------------------------------------

The **MOSEK** logo and name are trademarks of Mosek ApS.

The code is provided as-is. Compatibility with future release of **MOSEK** or the `Fusion API` are not guaranteed.

For more information contact [support_at_mosek.com](mailto:support@mosek.com)

---------------------------------------------------------------------------------------------------------------


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">The MOSEK collection of iPython Notebook</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="www.mosek.com" property="cc:attributionName" rel="cc:attributionURL">MOSEK ApS</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/MOSEK" rel="dct:source">https://github.com/MOSEK</a>.<br />Permissions beyond the scope of this license may be available at <a xmlns:cc="http://creativecommons.org/ns#" href="www.mosek.com" rel="cc:morePermissions">www.mosek.com</a>.